# SQL & Python

### create notebook to scrap data and save data in the database

In [3]:
from bs4 import BeautifulSoup
import requests
import sqlite3 

ln2=[]
ratedict={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}                          #dic for books rating
soup=BeautifulSoup(requests.get('https://books.toscrape.com/index.html').text,'html.parser')
connection=sqlite3.connect('bookscraper.db')
cat_link_list=[]
for i in soup.find('ul',attrs={'class':''}).find_all('li'):
    cat_link_list.append(i.find('a').get('href'))
for j in cat_link_list:
    soup=BeautifulSoup(requests.get('https://books.toscrape.com/'+ j ).text,'html.parser')
    n1=int(soup.find_all('strong')[1].get_text())              # no. of books in the category
    if n1 <= 20:       # does the category consists of one page or more, bec. in case of many pages the category link changes
        for k in soup.find_all('article', attrs={'class':'product_pod'}):
            Bookname=k.find('h3').find('a').get('title') # name of each book in the page
            Price=float(k.find('div',attrs={'class':'product_price'}).find('p',attrs={'class':'price_color'}).get_text()[2:])
            Rate=ratedict[k.find('p').get('class')[1]]
            Category=soup.find('li',attrs={'class':'active'}).get_text()
            cursor=connection.execute('insert into Books(Bookname,Price,Rate,Category) values(?,?,?,?)' , [Bookname,Price,Rate,Category])
            connection.commit()
    else:
        n2=int(soup.find('li',attrs={'class':'current'}).get_text().strip().split()[3]) #no. of pages in the categor
        for l in range(1,n2+1):
            soup=BeautifulSoup(requests.get('https://books.toscrape.com/catalogue/category/books/'+j.split('/')[3]+'/page-'+str(l)+'.html' ).text,'html.parser')
            ln2.append(j.split('/')[3])
            for k in soup.find_all('article', attrs={'class':'product_pod'}):
                Bookname=k.find('h3').find('a').get('title') # name of each book in the page
                Price=float(k.find('div',attrs={'class':'product_price'}).find('p',attrs={'class':'price_color'}).get_text()[2:])
                Rate=ratedict[k.find('p').get('class')[1]]
                Category=soup.find('li',attrs={'class':'active'}).get_text()
                cursor=connection.execute('insert into Books(Bookname,Price,Rate,Category) values(?,?,?,?)' , [Bookname,Price,Rate,Category]) 
                connection.commit()


In [14]:
connection.close()

### Create another notebook to answer some questions using SQL Language: 

In [26]:
# get books that has rate >=3 and has 'Mr' in its name
connection=sqlite3.connect('bookscraper.db')
rows=connection.execute('select * from Books where Rate >=? AND Bookname LIKE ? ' , [3,'%Mr%']).fetchall() 
rows

[(48, 'Mrs. Houdini', 30.25, 5, 'Historical Fiction'),
 (243, 'My Mrs. Brown', 24.48, 3, 'Fiction'),
 (248, 'Dear Mr. Knightley', 11.21, 5, 'Fiction')]

In [29]:
#books that has rate >=3 or price > 20
rows=connection.execute('select Bookname,Price,Rate from Books WHERE Rate>= ? OR Price > ?' , [3,20]).fetchall()
rows

[("It's Only the Himalayas", 45.17, 2),
 ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  49.43,
  4),
 ('See America: A Celebration of Our National Parks & Treasured Sites',
  48.87,
  3),
 ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  36.94,
  2),
 ('Under the Tuscan Sun', 37.33, 3),
 ('A Summer In Europe', 44.34, 2),
 ('The Great Railway Bazaar', 30.54, 1),
 ('A Year in Provence (Provence #1)', 56.88, 4),
 ('The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
  23.21,
  1),
 ('Neither Here nor There: Travels in Europe', 38.95, 3),
 ('1,000 Places to See Before You Die', 26.08, 5),
 ('Sharp Objects', 47.82, 4),
 ('The Past Never Ends', 56.5, 4),
 ('The Murder of Roger Ackroyd (Hercule Poirot #4)', 44.1, 4),
 ('The Last Mile (Amos Decker #2)', 54.21, 2),
 ('A Time of Torment (Charlie Parker #14)', 48.35, 5),
 ('Poisonous (Max Revere Novels #3)', 26.8, 3),
 ('Murder at the 42nd Str

In [32]:
# get books that has rate not 3.
rows=connection.execute('select * from Books WHERE Rate !=3').fetchall()
rows

[(1, "It's Only the Himalayas", 45.17, 2, 'Travel'),
 (2,
  'Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  49.43,
  4,
  'Travel'),
 (4,
  'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  36.94,
  2,
  'Travel'),
 (6, 'A Summer In Europe', 44.34, 2, 'Travel'),
 (7, 'The Great Railway Bazaar', 30.54, 1, 'Travel'),
 (8, 'A Year in Provence (Provence #1)', 56.88, 4, 'Travel'),
 (9,
  'The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
  23.21,
  1,
  'Travel'),
 (11, '1,000 Places to See Before You Die', 26.08, 5, 'Travel'),
 (12, 'Sharp Objects', 47.82, 4, 'Mystery'),
 (13, 'In a Dark, Dark Wood', 19.63, 1, 'Mystery'),
 (14, 'The Past Never Ends', 56.5, 4, 'Mystery'),
 (15, 'A Murder in Time', 16.64, 1, 'Mystery'),
 (16, 'The Murder of Roger Ackroyd (Hercule Poirot #4)', 44.1, 4, 'Mystery'),
 (17, 'The Last Mile (Amos Decker #2)', 54.21, 2, 'Mystery'),
 (18, 'That Darkness (Gardiner an

In [33]:
# get books that has price between 10 and 40 and has rate 3 or 4 or 1
rows=connection.execute('select * from Books WHERE (Price BETWEEN ? AND ?) AND (Rate IN (?,?,?))' , [10,40,3,4,1]).fetchall()
rows

[(5, 'Under the Tuscan Sun', 37.33, 3, 'Travel'),
 (7, 'The Great Railway Bazaar', 30.54, 1, 'Travel'),
 (9,
  'The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
  23.21,
  1,
  'Travel'),
 (10, 'Neither Here nor There: Travels in Europe', 38.95, 3, 'Travel'),
 (13, 'In a Dark, Dark Wood', 19.63, 1, 'Mystery'),
 (15, 'A Murder in Time', 16.64, 1, 'Mystery'),
 (18, 'That Darkness (Gardiner and Renner #1)', 13.92, 1, 'Mystery'),
 (19, 'Tastes Like Fear (DI Marnie Rome #3)', 10.69, 1, 'Mystery'),
 (22, 'Poisonous (Max Revere Novels #3)', 26.8, 3, 'Mystery'),
 (24, 'Most Wanted', 35.28, 3, 'Mystery'),
 (25, 'Hide Away (Eve Duncan #20)', 11.84, 1, 'Mystery'),
 (28, 'Playing with Fire', 13.71, 3, 'Mystery'),
 (31, 'Delivering the Truth (Quaker Midwife Mystery #1)', 20.89, 4, 'Mystery'),
 (32,
  'The Mysterious Affair at Styles (Hercule Poirot #1)',
  24.8,
  4,
  'Mystery'),
 (36, "The Cuckoo's Calling (Cormoran Strike #1)", 19.21, 1, 'Myster

In [38]:
# get the top 5 most expensive books (order books desc with price column and then limit the result to the first 5).
rows=connection.execute('select * from Books ORDER BY Price DESC LIMIT 5').fetchall()
rows

[(192, 'The Perfect Play (Play by Play #1)', 59.99, 3, 'Romance'),
 (272, 'Last One Home (New Beginnings #1)', 59.98, 3, 'Fiction'),
 (846, 'Civilization and Its Discontents', 59.95, 2, 'Psychology'),
 (926, 'The Barefoot Contessa Cookbook', 59.92, 5, 'Food and Drink'),
 (369, 'The Diary of a Young Girl', 59.9, 3, 'Nonfiction')]

In [64]:
# get the get the 3rd 10 books in the books table order first by rate desc and then by price asc.
rows=connection.execute('select * from Books ORDER BY rate DESC , Price ASC LIMIT 20,10' ).fetchall()
rows

[(763, 'The Epidemic (The Program 0.6)', 14.44, 5, 'Young Adult'),
 (189,
  "A Gentleman's Position (Society of Gentlemen #3)",
  14.75,
  5,
  'Romance'),
 (862,
  'Hyperbole and a Half: Unfortunate Situations, Flawed Coping Mechanisms, Mayhem, and Other Things That Happened',
  14.75,
  5,
  'Humor'),
 (424,
  'Disrupted: My Misadventure in the Start-Up Bubble',
  15.28,
  5,
  'Nonfiction'),
 (858, 'Fifty Shades Freed (Fifty Shades #3)', 15.36, 5, 'Adult Fiction'),
 (94, 'Batman: The Dark Knight Returns (Batman)', 15.38, 5, 'Sequential Art'),
 (828,
  'The Collected Poems of W.B. Yeats (The Collected Works of W.B. Yeats #1)',
  15.42,
  5,
  'Poetry'),
 (780, 'Kill the Boy Band', 15.52, 5, 'Young Adult'),
 (164, "Sophie's World", 15.94, 5, 'Philosophy'),
 (546, 'Shiver (The Wolves of Mercy Falls #1)', 16.23, 5, 'Default')]

In [44]:
# add a new category in categories table and add 5 books to this category in books table with title, rate and price
cursor=connection.execute('INSERT INTO Category_table(Category) VALUES (?)' , ["New Category"])
cursor=connection.execute('INSERT INTO Books(Bookname,Price,Rate,Category) Values(?,?,?,?),(?,?,?,?),(?,?,?,?),(?,?,?,?),(?,?,?,?)' , ["Newbook1",10,1,"Newcategory","Newbook2",20,2,"Newcategory","Newbook3",30,3,"Newcategory","Newbook4",40,4,"Newcategory","Newbook5",50,5,"Newcategory"])
connection.commit()

In [46]:
# update book's rate to 3 that thier price < 20£
cursor=connection.execute('UPDATE Books SET Rate=? WHERE Price < ?' , [3,20])
connection.commit()

In [47]:
# delete all books that have price > 50£ and has rate <= 2.
cursor=connection.execute('DELETE FROM BOOKS WHERE PRICE>? AND RATE<=?' , [50,2])
connection.commit()

In [54]:
# count the number of books that have 'Secret' in thier names and price between 10£ and 25£
cursor=connection.execute('SELECT COUNT (Bookname) From Books WHERE (BookName LIKE ?) AND (Price BETWEEN ? AND ?)' , ["%SECRET%",10,25]).fetchall()
cursor

[(5,)]

In [59]:
# get the minimum & maximum price for all the books that have rate 5
cursor1=connection.execute('SELECT MIN (Price) FROM Books WHERE Rate=5').fetchall()
cursor2=connection.execute('SELECT MAX (Price) FROM Books WHERE Rate=5').fetchall()
print(cursor1,cursor2)

[(20.02,)] [(59.92,)]


In [62]:
# calculate the avg price for all the books that have rate 5
cursor=connection.execute('SELECT AVG(Price) FROM Books WHERE Rate=5').fetchall()
cursor[0][0]

41.10774193548389

In [64]:
# sum all book's price that have rate 2 and price between 10 and 40
cursor=connection.execute('SELECT SUM(Price) FROM Books WHERE Price BETWEEN 10 AND 40').fetchall()
cursor[0][0]


15055.810000000005

In [66]:
# join both books & categories table into one new table containing book_name & category_name & book_rate and book_price.
cursor=connection.execute('SELECT Books.Bookname, Category_table.Category FROM Books LEFT JOIN Category_table ON Books.Category=Category_table.Category').fetchall()
cursor

[("It's Only the Himalayas", 'Travel'),
 ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond',
  'Travel'),
 ('See America: A Celebration of Our National Parks & Treasured Sites',
  'Travel'),
 ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel',
  'Travel'),
 ('Under the Tuscan Sun', 'Travel'),
 ('A Summer In Europe', 'Travel'),
 ('The Great Railway Bazaar', 'Travel'),
 ('A Year in Provence (Provence #1)', 'Travel'),
 ('The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)',
  'Travel'),
 ('Neither Here nor There: Travels in Europe', 'Travel'),
 ('1,000 Places to See Before You Die', 'Travel'),
 ('Sharp Objects', 'Mystery'),
 ('In a Dark, Dark Wood', 'Mystery'),
 ('The Past Never Ends', 'Mystery'),
 ('A Murder in Time', 'Mystery'),
 ('The Murder of Roger Ackroyd (Hercule Poirot #4)', 'Mystery'),
 ('That Darkness (Gardiner and Renner #1)', 'Mystery'),
 ('Tastes Like Fear (DI Marnie Rome #3)', 'Mystery

In [76]:
# calculate how many books each rate has and have price between 20£ and 30£.

cursor=connection.execute('SELECT Rate , COUNT (Bookname) FROM Books WHERE Price BETWEEN 20 AND 30 GROUP BY Rate ').fetchall()
cursor

[(1, 56), (2, 41), (3, 42), (4, 34), (5, 36)]

In [77]:
# calculate how many books each category has having count > 10

cursor=connection.execute('SELECT Category , COUNT (Bookname) as bookscount FROM Books GROUP BY Category Having bookscount >10 ').fetchall()
cursor

[('Add a comment', 60),
 ('Business', 11),
 ('Childrens', 28),
 ('Classics', 17),
 ('Default', 144),
 ('Fantasy', 40),
 ('Fiction', 60),
 ('Food and Drink', 28),
 ('Historical Fiction', 24),
 ('History', 16),
 ('Horror', 15),
 ('Music', 11),
 ('Mystery', 30),
 ('Nonfiction', 104),
 ('Poetry', 18),
 ('Romance', 33),
 ('Science', 13),
 ('Science Fiction', 14),
 ('Sequential Art', 70),
 ('Thriller', 11),
 ('Travel', 11),
 ('Womens Fiction', 16),
 ('Young Adult', 50)]

In [78]:
# get all books with category_name='Music' using subquery

cursor=connection.execute('SELECT * FROM Books WHERE Category=(SELECT Category FROM Category_table WHERE Category="Music") ').fetchall()
cursor

[(438, 'Rip it Up and Start Again', 35.02, 5, 'Music'),
 (439,
  'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
  57.25,
  3,
  'Music'),
 (440, 'How Music Works', 37.32, 2, 'Music'),
 (441, 'Love Is a Mix Tape (Music #1)', 18.03, 3, 'Music'),
 (442,
  'Please Kill Me: The Uncensored Oral History of Punk',
  31.19,
  4,
  'Music'),
 (443,
  "Kill 'Em and Leave: Searching for James Brown and the American Soul",
  45.05,
  5,
  'Music'),
 (445,
  'This Is Your Brain on Music: The Science of a Human Obsession',
  38.4,
  1,
  'Music'),
 (446,
  'Orchestra of Exiles: The Story of Bronislaw Huberman, the Israel Philharmonic, and the One Thousand Jews He Saved from Nazi Horrors',
  12.36,
  3,
  'Music'),
 (447, 'No One Here Gets Out Alive', 20.02, 5, 'Music'),
 (448, 'Life', 31.58, 5, 'Music'),
 (450, 'Forever Rockers (The Rocker #12)', 28.8, 3, 'Music')]

In [87]:
# Export Database to CSV File and use pandas to read this CSV 

import pandas as pd
df=pd.read_sql('SELECT * FROM Books',connection)
df.to_csv('Books.csv')
df2=pd.read_csv('Books.csv')
df2

,Unnamed: 0,ID,Bookname,Price,Rate,Category
0,0,1,It's Only the Himalayas,45.17,2,Travel
1,1,2,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,Travel
2,2,3,See America: A Celebration of Our National Par...,48.87,3,Travel
3,3,4,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,Travel
4,4,5,Under the Tuscan Sun,37.33,3,Travel
...,...,...,...,...,...,...
926,926,1001,Newbook1,10.00,3,Newcategory
927,927,1002,Newbook2,20.00,2,Newcategory
928,928,1003,Newbook3,30.00,3,Newcategory
929,929,1004,Newbook4,40.00,4,Newcategory
